# Artificial and Computational Intelligence Assignment 1

## Problem solving by Uninformed & Informed Search

List only the BITS (Name) of active contributors in this assignment:
1. ___________________
2. __________________
3. ____________________
4. ___________________
5. ___________________

Things to follow
1.	Use appropriate data structures to represent the graph and the path using python libraries
2.	Provide proper documentation
3.	Find the path and print it

Coding begins here

### 1.	Define the environment in the following block

List the PEAS decription of the problem here in this markdown block


Design the agent as PSA Agent(Problem Solving Agent)
Clear Initial data structures to define the graph and variable declarations is expected
IMPORTATANT: Write distinct code block as below

In [2]:
#Code Block : Set Initial State (Must handle dynamic inputs)
class GridGenerator:
    def __init__(self):
        self.grid = [
            ['.', '.', '.', '.', '#', '#', '.', '.'],
            ['.', 'F', 'F', '.', '.', '.', '.', 'F'],
            ['.', '.', '.', '.', '.', '.', '.', '.'],
            ['#', '#', '.', 'F', '.', '.', '.', '.'],
            ['.', 'F', '.', '.', '.', '.', '#', '#'],
            ['.', '.', '.', 'F', '.', '.', '.', '.'],
            ['.', '#', '#', '.', '.', 'F', '.', '.'],
            ['.', '.', '.', '.', '.', 'F', '.', '.']
        ]

    def generate_grid(self):
        def set_position(position, symbol):
            row, col = position
            if row < 0 or row > len(self.grid) or col < 0 or col > len(self.grid[0]):
                return None, "Please enter rows position between 0 to {rows} and columns between 0 to {cols}".format(rows=len(self.grid - 1), cols=len(self.grid[0] - 1))

            if self.grid[row][col] == '#':
                self.print_grid()
                return None, f"Invalid {symbol} position! This cell is occupied by a water body ('#')."

            if self.grid[row][col] == 'F':
                self.print_grid()
                return None, f"Invalid {symbol} position! This cell is flooded ('F')."

            if self.grid[row][col] == 'S':
                self.print_grid()
                return None, f"Invalid {symbol} position! The start and the Goal position cannot be same."

            self.grid[row][col] = symbol

            return self.grid, None

        # Start position input and validation
        while True:
            start_input = input("\nEnter the start position (row,col): ")
            start_row, start_col = map(int, start_input.split(','))
            grid_with_start, error_message = set_position((start_row, start_col), 'S')
            if error_message:
                print(error_message)
            else:
                break

        # Goal position input and validation
        while True:
            goal_input = input("\nEnter the goal position (row,col): ")
            goal_row, goal_col = map(int, goal_input.split(','))
            grid_with_goal, error_message = set_position((goal_row, goal_col), 'G')
            if error_message:
                print(error_message)
            else:
                break

        # Print final grid with start and goal positions
        print("\nFinal grid with start and goal positions:")
        self.print_grid()

    def print_grid(self):
        for row in self.grid:
            print(' '.join(row))




In [4]:
#Code Block : Set the matrix for transition & cost (as relevant for the given problem)

import time
import platform

if platform.system() == "Windows":
    import psutil
elif platform.system() == "Darwin" or platform.system() == "Linux":
    import resource

if platform.system() == "Windows":
    def track_time_and_space(func):
        def wrapper(*args, **kwargs):
            start_time = time.time()
            start_memory = psutil.Process().memory_info().rss 

            result = func(*args, **kwargs)
            
            end_time = time.time()
            end_memory = psutil.Process().memory_info().rss

            print(platform.system(), start_memory, end_memory)

            execution_time = end_time - start_time
            memory_usage = (end_memory - start_memory) / 1024  # Convert to kilobytes

            print(f"Execution time: {execution_time} seconds")
            print(f"Memory usage: {memory_usage} KB")

            return result, execution_time, memory_usage

        return wrapper
else:
    def track_time_and_space(func):
        def wrapper(*args, **kwargs):
            start_time = time.time()
            start_memory = resource.getrusage(resource.RUSAGE_SELF).ru_maxrss

            result = func(*args, **kwargs)
            
            end_time = time.time()
            end_memory = resource.getrusage(resource.RUSAGE_SELF).ru_maxrss

            execution_time = end_time - start_time
            memory_usage = (end_memory - start_memory) / 1024  # Convert to kilobytes

            print(f"Execution time: {execution_time} seconds")
            print(f"Memory usage: {memory_usage} KB")

            return result, execution_time, memory_usage
        
        return wrapper




In [ ]:
#Code Block : Write function to design the Transition Model/Successor function. Ideally this would be called while search algorithms are implemented
import matplotlib.pyplot as plt
import random
import sys

class GridEnvironment:
    def __init__(self, grid: list[list[str]], display: bool = False):
        def find_start_and_goal():
            for i in range(self.rows):
                for j in range(self.cols):
                    if self.grid[i][j] == 'S':
                        self.start = tuple((i, j))
                    elif self.grid[i][j] == 'G':
                        self.goal = tuple((i, j))
            print("Start {start}, Goal {goal}:".format(start=self.start, goal=self.goal))

        self.grid = grid
        self.rows = len(self.grid)
        self.cols = len(self.grid[0])
        self.display = display
        find_start_and_goal()

    def visualize(self, path):
         # Define colors for different elements
        colors = {'S': 'green', '.': 'white', '#': 'black', 'F': 'red', 'G': 'blue'}
        # Create a plot
        fig, ax = plt.subplots()

        # Plot each element with specified colors      
        for i in range(self.rows):
            for j in range(self.cols):
                ax.text(i, j, self.grid[i][j], ha='center', va='center', color=colors[self.grid[i][j]])

        # Customize ticks
        ax.set_xticks(range(len(self.grid[0])))
        ax.set_yticks(range(len(self.grid)))
        ax.set_xticklabels([])
        ax.set_yticklabels([])
        
        plt.title('S - Start, G - Goal')
        plt.xlabel('# - Water Bodies')
        plt.ylabel('F - Flooded Roads')

        # Add grid lines
        ax.grid(True, which='both', color='black', linewidth=1.5, linestyle='-', alpha=0.3)
        
        col = ['red', 'green', 'blue', 'cyan', 'magenta', 'orange', 'purple']
        colors = [col[random.randint(0, 6)] for _ in range(sys.getsizeof(path)-1)]
        X, Y = zip(*path)
        
        # Plot the path with different colors for each segment
        for i in range(sys.getsizeof(path)-1):
            plt.plot(X[i:i+2], Y[i:i+2], color=colors[i])
        
        # plt.plot(*zip(*path), marker='o', color='red', label='Path')
        plt.scatter(*path[0], marker='o', color='green', label='Start')
        plt.scatter(*path[-1], marker='o', color='blue', label='Goal')

    def is_valid_move(self, row, col):
        return (
            0 <= row < self.rows 
            and 0 <= col < self.cols 
            and self.grid[row][col] != '#' 
            and self.grid[row][col] != 'F'
        )

    def get_adjacent_cells(self, row, col, algorithm=None):
        directions = [(1, 0), (-1, 0), (0, 1), (0, -1)] #degrees of freedom - need to revise by prasnejit
        adjacent_cells = [] # Return empty list of adjacent cells if all moves are invalid
        for dr, dc in directions:
            new_row, new_col = row + dr, col + dc
            if self.is_valid_move(new_row, new_col):
                if algorithm == "greedy" and self.display:
                    print(" The move is valid for this %d and %d" % (new_row, new_col))
                adjacent_cells.append(tuple((new_row, new_col)))
        return adjacent_cells
    
    def goal_reached(self, row, col):
        return (row, col) == self.goal


In [5]:
#Code block : Write fucntion to handle goal test (Must handle dynamic inputs). Ideally this would be called while search algorithms are implemented
def goal_reached(self, row, col):
        return (row, col) == self.goal

### 2.	Definition of Algorithm 1 (Mention the Name of the algorithm here)

In [6]:
#Code Block : Function for algorithm 1 implementation
from heapq import heappush, heappop
from collections import deque
from typing import List, Tuple

class GBFSearchAlgorithm(ISearchAlgorithm):
    """
    Greedy Best-First Search Algorithm implementation.

    This algorithm uses a priority queue to explore the search space based on a heuristic function.
    It expands the node with the lowest heuristic value, prioritizing the most promising paths towards the goal.

    Attributes:
        None

    Methods:
        search(grid_env: GridEnvironment) -> Tuple[List[Tuple[int, int]], int]:
            Performs the greedy best-first search on the given grid environment.

        heuristic(row: int, col: int) -> int:
            Calculates the heuristic value for a given cell in the grid.

        time_complexity() -> str:
            Returns the time complexity of the algorithm.

        space_complexity() -> str:
            Returns the space complexity of the algorithm.

    """
    def __init__(self, grid_env: GridEnvironment) -> None:
        super().__init__()

        self.grid_env = grid_env
        self.tree = {}

    @track_time_and_space
    def search(self) -> Tuple[List[Tuple[int, int]], int, int]:
        """
        Performs the greedy best-first search on the given grid environment.

        Args:
            grid_env (GridEnvironment): The grid environment to search in.

        Returns:
            Tuple[List[Tuple[int, int]], int]: A tuple containing the path taken by the agent and the total path cost.

        """
        start = self.grid_env.start
        goal = self.grid_env.goal
        visited = set()
        pq = [(self.heuristic(*start), start)]
        came_from = {}
        cost_so_far = {start: 0}  # Store the cost of reaching each cell
        total_nodes_expanded = 0
        total_branching_factor = 0
        depth_of_solution = 0

        while pq:
                print("Open List (Priority Queue):", pq)
                print("Closed List (Visited Set):", visited)
                priority, current = heappop(pq)  # Pop the item with lowest priority
                if current == goal:
                    # Reconstruct the path
                    path = deque()
                    total_cost = cost_so_far[current]
                    while current != start:
                        path.appendleft(current)
                        current = came_from[current]
                        depth_of_solution += 1  # Increment depth for each step towards the start node
                    path.appendleft(start)
                    break  # Exit the loop when the goal node is found
                    
                visited.add(current)
                total_nodes_expanded += 1

                successors = self.grid_env.get_adjacent_cells(*current, algorithm="greedy")
                total_branching_factor += len(successors)

                for next_cell in successors:
                    if next_cell in visited:  # Check if the cell has already been visited
                        continue  # Skip to the next iteration if the cell has been visited

                    new_cost = cost_so_far[current] + 1  # Assuming each step has a cost of 1
                    if next_cell not in cost_so_far or new_cost < cost_so_far[next_cell]:
                        cost_so_far[next_cell] = new_cost
                        # print("Cost of next cell", cost_so_far[next_cell])
                        priority = new_cost + self.heuristic(*next_cell)
                        print("Priority of next cell", priority)
                        heappush(pq, (priority, next_cell))  # Add the next cell to the priority queue
                        came_from[next_cell] = current

            # Store the depth of the optimal solution
        self.depth_of_solution = depth_of_solution
        print("Total Branching Factor:", round(total_branching_factor / total_nodes_expanded))
        print("Depth of the graph search tree is:", self.depth_of_solution)
            # Optionally, you can return the path and total cost if needed
        print("Space Complexity for GBFS Search is :", pq.count)
        return list(path), total_cost
    
 

    def heuristic(self, row: int, col: int) -> int:
        """
        Calculates heuristic value for a given cell in the grid. Considering
        if the adjacent cells are safe, water bodies, or flooded roads

        Args:
            row (int): The row index of the cell.
            col (int): The column index of the cell.

        Returns:
            int: The heuristic value for the cell.

        """
        score = 0
        for dr, dc in [(1, 0), (-1, 0), (0, 1), (0, -1)]:
            new_row, new_col = row + dr, col + dc
            if 0 <= new_row < self.grid_env.rows and 0 <= new_col < self.grid_env.cols:
                if self.grid_env.grid[new_row][new_col] == '.':
                    score += 5  # Add 5 points for adjacent safe places
                elif self.grid_env.grid[new_row][new_col] == '#':
                    score -= 5  # Deduct 5 points for adjacent water bodies
                elif self.grid_env.grid[new_row][new_col] == 'F':
                    score -= 3  # Deduct 3 points for flooded roads
        return score



NameError: name 'ISearchAlgorithm' is not defined

### 3.	Definition of Algorithm 2 (Mention the Name of the algorithm here)

In [ ]:
#Code Block : Function for algorithm 2 implementation
import random
from abc import ABC

class Individual(ABC):
    def __init__(self, grid_env: GridEnvironment):
        self.individual_type = "Child"  # Every individual is a child, until it is a parent
        self.grid_env = grid_env
        self.path = [tuple(grid_env.start)]
        self.fitness = 0

        current_pos = grid_env.start
        while current_pos != grid_env.goal:
            adjacent_cells = grid_env.get_adjacent_cells(*current_pos, algorithm="genetic")
            next_pos = random.choice(adjacent_cells)
            self.path.append(next_pos)
            current_pos = next_pos

    def set_path(self, path: list):
        self.path = path
    
    def evaluate_fitness(self):
        for pos in self.path:
            row, col = pos
            if self.grid_env.grid[row][col] == '.':
                self.fitness += 5
            elif self.grid_env.grid[row][col] == '#':
                self.fitness -= 5
            elif self.grid_env.grid[row][col] == 'F':
                self.fitness -= 3

    def mutate(self, mutation_rate: float):
        for i in range(1, len(self.path) - 1):
            if random.random() < mutation_rate:
                adjacent_cells = self.grid_env.get_adjacent_cells(*self.path[i], algorithm="genetic")
                self.path[i] = random.choice(adjacent_cells)

class GeneticSearchAlgorithm(ISearchAlgorithm):
    def __init__(self, grid_env):
        self.grid_env: GridEnvironment = grid_env
        self.population_size = 10
        self.generations = 100
        self.mutation_rate = 0.01

    def crossover(self, parent1, parent2):
        crossover_point = random.randint(1, min(len(parent1.path), len(parent2.path)) - 1)
        # Single point crossover
        child_path = parent1.path[:crossover_point] + parent2.path[crossover_point:]
        
        child = Individual(self.grid_env)
        child.set_path(child_path)
        
        return child

    @track_time_and_space
    def search(self):
        population = [Individual(self.grid_env) for _ in range(self.population_size)]

        for generation in range(self.generations):
            for individual in population:
                individual.evaluate_fitness()

            population.sort(key=lambda x: x.fitness, reverse=True)

            if population[0].fitness == 40:  # Max possible fitness - to be revised later 
                break

            next_generation = population[:2]  # Elitism
            while len(next_generation) < self.population_size:
                parent1 = random.choice(population[:self.population_size // 2])
                parent2 = random.choice(population[:self.population_size // 2])
                child = self.crossover(parent1, parent2)
                child.mutate(self.mutation_rate)
                next_generation.append(child)

            population = next_generation

        best_individual = population[0]
        path = best_individual.path
        cost = 0
        for pos in path:
            row, col = pos
            if self.grid_env.grid[row][col] == '.':
                cost += 5
            elif self.grid_env.grid[row][col] == '#':
                cost -= 5
            elif self.grid_env.grid[row][col] == 'F':
                cost -= 3
        return path, cost


### DYNAMIC INPUT

IMPORTANT : Dynamic Input must be got in this section. Display the possible states to choose from:
This is applicable for all the relevent problems as mentioned in the question.

In [ ]:
#Code Block : Function & call to get inputs (start/end state)


### 4.	Calling the search algorithms
(For bidirectional search in below sections first part can be used as per Hint provided. Under second section other combinations as per Hint or your choice of 2 algorithms can be called .As an analyst suggest suitable approximation in the comparitive analysis section)

In [ ]:
#invoke of algorithm 
from pprint import pprint

from utils.GridGenerator import GridGenerator
from utils.GridEnvironment import GridEnvironment
from Algorithms.GBFSearchAlgorithm import GBFSearchAlgorithm
from Algorithms.GeneticSearchAlgorithm import GeneticSearchAlgorithm

class SearchAlgorithmFactory:
    @staticmethod
    def create_search_algorithm(args, env: GridEnvironment):
        if not env:
            grid_gen = GridGenerator()
            grid_gen.generate_grid()
            env = GridEnvironment(grid_gen.grid, args.display)

        if args.display:
            pprint(env.grid) # Print the grid

        if args.gbfs:
            return GBFSearchAlgorithm(env)
        elif args.genetic:
            return GeneticSearchAlgorithm(env)
        else:
            print("Please select an algorithm to run. Use -h for help.")
            return


In [ ]:
#Invoke algorithm 1 (Should Print the solution, path, cost etc., (As mentioned in the problem))
import argparse
from pprint import pprint
from datetime import datetime

from utils.GridGenerator import GridGenerator
from utils.GridEnvironment import GridEnvironment
from utils.PersistPerformance import PersistPerformance
from utils.PlotPerformance import PlotPerformance
from Algorithms.GBFSearchAlgorithm import GBFSearchAlgorithm
from Algorithms.GeneticSearchAlgorithm import GeneticSearchAlgorithm
from Algorithms.SearchAlgorithmFactory import SearchAlgorithmFactory

def main():
    """
    This script runs a search algorithm on a grid environment.

    Usage:
        python main.py [-a | -b] [-d]

    Options:
        -a, --genetic   Run Genetic Algorithm
        -b, --gbfs      Run Greedy Best First Search
        -d, --display   Display grid

    Example:
        python main.py          ## Run both Genetic Algorithm and Greedy Best First Search
        python main.py -b -d    ## Run Greedy Best First Search
        python main.py -a       ## Run Genetic Algorithm
    """

    parser = argparse.ArgumentParser()
    group = parser.add_mutually_exclusive_group()
    group.add_argument("-a", "--genetic", action="store_true", help="Run Genetic Algorithm")
    group.add_argument("-b", "--gbfs", action="store_true", help="Run Greedy Best First Search")
    parser.add_argument("-d", "--display", action="store_true", help="Display grid")
    args = parser.parse_args()

    dt_now = datetime.now()

    if(args.genetic == False) and (args.gbfs == False):
        print("Running both Genetic Algorithm and Greedy Best First Search")

        args.gbfs = True
        gbfs = SearchAlgorithmFactory.create_search_algorithm(args, None)
        gbfs_results, gbfs_etime, gbfs_emem = gbfs.search()
        gbfs_path = gbfs_results[0]
        gbfs_cost = gbfs_results[1]

        print("Path taken by the agent using Greedy Best First Search:", gbfs_path)
        print("Total path cost using Greedy Best First Search:", gbfs_cost)
        args.gbfs = False

        pp_gbfs = PersistPerformance(
            date_time=dt_now,
            execution_time=gbfs_etime,
            memory_usage=gbfs_emem,
            grid_shape=(gbfs.grid_env.rows, gbfs.grid_env.cols),
            start=gbfs.grid_env.start,
            goal=gbfs.grid_env.goal,
            algorithm="Greedy Best First Search"
        )
        # pp_gbfs.persist()
        pprint(pp_gbfs)

        args.genetic = True
        genetic_search = SearchAlgorithmFactory.create_search_algorithm(args, gbfs.grid_env.grid)
        gs_results, gs_etime, gs_emem = genetic_search.search()
        gs_path = gs_results[0]
        gs_cost = gs_results[1]

        print("Path taken by the agent using Genetic Algorithm:", gs_path)
        print("Total path cost using Genetic Algorithm:", gs_cost)

        pp_gs = PersistPerformance(
            date_time=dt_now,
            execution_time=gs_etime,
            memory_usage=gs_emem,
            grid_shape=(genetic_search.grid_env.rows, genetic_search.grid_env.cols),
            start=genetic_search.grid_env.start,
            goal=genetic_search.grid_env.goal,
            algorithm="Genetic Search"
        )
        # pp_gs.persist()
        pprint(pp_gs)


    else:
        search_algorithm = SearchAlgorithmFactory.create_search_algorithm(args, None)
        
        search_results, etime, emem = search_algorithm.search()
        path = search_results[0]
        cost = search_results[1]

        print("Path taken by the agent:", path)
        print("Total path cost:", cost)

        pp = PersistPerformance(
            date_time=dt_now,
            execution_time=etime,
            memory_usage=emem,
            grid_shape=(search_algorithm.grid_env.rows, search_algorithm.grid_env.cols),
            start=(0, 0),
            goal=(7, 7),
            algorithm="Genetic Search" if args.genetic else "Greedy Best First Search"
        )
        pp.persist()

        gbfs_records = pp.fetch_by("algorithm", "Greedy Best First Search")

        grid_size_list = []
        exe_time_list = []
        mem_con_list = []

        for data in gbfs_records:
            for key, value in data.items():
                if key == "grid_size":
                    grid_size_list.append(value[0])
                elif key == "execution_time":
                    exe_time_list.append(value)
                elif key == "memory_usage":
                    mem_con_list.append(value)

        # Create an instance of PlotPerformance
        plot_instance = PlotPerformance(gridSize=grid_size_list, timeConsumed=exe_time_list, memoryConsumed=mem_con_list)

        # Call the plot method
        plot_instance.plot()
    

if __name__ == "__main__":
    main()


#invoke of algorithm 
python main.py -b -d 

In [ ]:
#Invoke algorithm 2 (Should Print the solution, path, cost etc., (As mentioned in the problem))
#invoke of algorithm 
python main.py -a -d 

### 5.	Comparitive Analysis

In [ ]:
#Code Block : Print the Time & Space complexity of algorithm 1

In [ ]:
#Code Block : Print the Time & Space complexity of algorithm 2

### 6.	Provide your comparitive analysis or findings in no more than 3 lines in below section

Comparison : _______________________________________________

________________________________________________________

_________________________________________________________